# Read results processed by MintPy

## functions

In [1]:
import os
import h5py
import random
import numpy as np
import matplotlib.pyplot as plt
from xml.dom.minidom import parse
import zipfile
%matplotlib inline


def read_h5(fname, label):
    with h5py.File(fname, 'r') as f:
        atr = dict(f.attrs)
        data = np.asarray(f[(label)])
    return data, atr


def read_vel(vel_file, mask_file, ds_factor=1, out_vel_file=None):
    vel, atr = read_h5(vel_file, 'velocity')

    lon = float(atr['X_FIRST'])
    lat = float(atr['Y_FIRST'])

    lon_step = float(atr['X_STEP'])
    lat_step = float(atr['Y_STEP'])

    lon_tmp = (np.linspace(lon, lon + lon_step * vel.shape[1], vel.shape[1]))
    lat_tmp = (np.linspace(lat, lat + lat_step * vel.shape[0], vel.shape[0]))
    lons, lats = np.meshgrid(lon_tmp, lat_tmp)

    mask, _ = (read_h5(mask_file, 'mask'))

    vel = vel[::ds_factor, ::ds_factor]
    mask = mask[::ds_factor, ::ds_factor]
    lons = lons[::ds_factor, ::ds_factor]
    lats = lats[::ds_factor, ::ds_factor]

    lons = lons[mask].reshape((-1, 1))
    lats = lats[mask].reshape((-1, 1))
    vel = vel[mask].reshape((-1, 1))

    num = np.arange(vel.shape[0]).reshape((-1, 1))

    vel *= 1000

    print('max velocity : ', np.max(vel))
    print('min velocity : ', np.min(vel))
    print('number of points : ', vel.shape[0])

    out_data = np.hstack((num, lons, lats, vel))

    if not out_vel_file is None:
        print('writing data to {}'.format(out_vel_file))
        np.savetxt(out_vel_file, out_data, fmt='%4f')
        print('done.')


def read_vel_ts(ts_file,
                vel_file,
                mask_file,
                ds_factor=1,
                out_vel_file=None,
                out_ts_file=None):
    mask, _ = read_h5(mask_file, 'mask')
    mask = np.asarray(mask)

    vel, _ = read_h5(vel_file, 'velocity')
    vel = np.asarray(vel) * 1000

    date, _ = read_h5(ts_file, 'date')
    date = date.astype(np.int64)
    ts, atr = read_h5(ts_file, 'timeseries')
    ts = np.asarray(ts)
    ts = ts.reshape((date.shape[0], vel.shape[0], vel.shape[1])) * 1000
    ts = ts[:, ::ds_factor, ::ds_factor]
    ts = ts.reshape((date.shape[0], -1, 1))

    lon = float(atr['X_FIRST'])
    lon_step = float(atr['X_STEP'])

    lat = float(atr['Y_FIRST'])
    lat_step = float(atr['Y_STEP'])

    lon_tmp = (np.linspace(lon, lon + lon_step * vel.shape[1], vel.shape[1]))
    lat_tmp = (np.linspace(lat, lat + lat_step * vel.shape[0], vel.shape[0]))

    lons, lats = np.meshgrid(lon_tmp, lat_tmp)
    
    lons = lons[::ds_factor, ::ds_factor]
    lats = lats[::ds_factor, ::ds_factor]
    mask = mask[::ds_factor, ::ds_factor]
    vel = vel[::ds_factor, ::ds_factor]

    lons = lons.reshape((-1, 1))
    lats = lats.reshape((-1, 1))
    vels = vel.reshape((-1, 1))
    mask = mask.reshape((-1, 1))

    lons = lons[mask].reshape((-1, 1))
    lats = lats[mask].reshape((-1, 1))
    vels = vels[mask].reshape((-1, 1))
    num = np.arange(lons.shape[0]).reshape((-1, 1))

    print('number of points : ', lons.shape[0])
    print('max velocity : ', np.max(vels))
    print('min velocity : ', np.min(vels))
    out_vel = np.hstack((num, lons, lats, vels))

    out_ts1 = out_vel
    for i in range(ts.shape[0]):
        data = ts[i]
        out_ts1 = np.hstack((out_ts1, data[mask].reshape((-1, 1))))

    tmp = out_ts1[:, 4:]
    tmp = tmp - tmp[:, 0].reshape((-1, 1))
    tmp = np.hstack((out_vel, tmp))

    out_ts1 = tmp
    print('max cumulative displacement: ', np.max(out_ts1[:, -1]))
    print('min cumulative displacement: ', np.min(out_ts1[:, -1]))

    first_line = np.asarray([[-1, -1, -1, -1]])
    first_line = np.hstack((first_line, date.reshape((1, -1))))
    out_ts = np.vstack((first_line, out_ts1))
    out_vel = out_ts1[0:, 0:4]

    if not out_ts_file is None:
        print('writing data to {}'.format(out_ts_file))
        np.savetxt(out_ts_file, out_ts, fmt='%4f')
        print('done.')

    if not out_vel_file is None:
        print('writing data to {}'.format(out_vel_file))
        np.savetxt(out_vel_file, out_vel, fmt='%4f')
        print('done.')

    return out_ts, date


def date2str(date):
    date_str = [str(i) for i in date]
    return date_str


def plot_displacement(num_list,
                      ts_data,
                      date,
                      aspect=0.2,
                      figsize=(15, 7),
                      y_lim=[-100, 100],
                      fig_name=None):
    fig, ax = plt.subplots(figsize=figsize)
    ax.set_title('time series displacement', fontsize=40, pad=20)
    ax.set_xlabel('date', fontsize=30, labelpad=10)
    ax.set_ylabel('displacrment (mm)', fontsize=30, labelpad=10)

    ax.set_ylim(y_lim[0], y_lim[1])
    ax.set_aspect(aspect)
    ax.minorticks_on()
    ax.xaxis.grid(True, which='major')
    ax.xaxis.set_tick_params(rotation=30, labelsize=15)
    ax.yaxis.grid(True, which='major')
    ax.yaxis.set_tick_params(rotation=0, labelsize=15)
    ax.set_xmargin(0.02)

    date = date2str(date)

    for num in num_list:
        disp = ts_data[num, 4:]
        ax.plot(date, disp, label=str(num), marker='o')
        ax.xaxis.set_ticks(date[::4])
        ax.yaxis.set_ticks(list(range(y_lim[0], y_lim[1] + 10, 10)))
    ax.legend(loc='best', fontsize=20, ncol=2)
    fig.show()
    if not fig_name is None:
        fig.savefig(fig_name, dpi=200)


def intersect(point, s_point, e_point):
    # parallel and coincident with the ray，s_point coincides with s_point
    if s_point[1] == e_point[1]:
        return False
    # line segment is above the ray
    if s_point[1] > point[1] and e_point[1] > point[1]:
        return False
    # line segment under the ray
    if s_point[1] < point[1] and e_point[1] < point[1]:
        return False
    # point coincides with s_point
    if s_point[1] == point[1] and e_point[1] > point[1]:
        return False
    # point coincides with e_point
    if e_point[1] == point[1] and s_point[1] > point[1]:
        return False
    # line segment is to the left of the ray
    if s_point[0] < point[0] and e_point[0] < point[0]:
        return False
    # find the intersection
    xseg = e_point[0] - (e_point[0] - s_point[0]) * (e_point[1] - point[1]) / (
        e_point[1] - s_point[1])
    # intersection is to the left of point
    if xseg < point[0]:
        return False
    return True


def inpolygon(point, polygon):
    num = 0  # number of intersection
    for i in range(len(polygon) - 1):
        if intersect(point, polygon[i], polygon[i + 1]):
            num += 1
    return True if num % 2 == 1 else False


def kml2polygon_dict(kml_file):
    # unzip kmz to get kml
    doc_kml = ''
    if kml_file.endswith('.kmz'):
        dir_name = os.path.dirname(kml_file)
        with zipfile.ZipFile(kml_file, 'r') as f:
            files = f.namelist()
            f.extract(files[0], dir_name)
        doc_kml = os.path.join(dir_name, 'doc.kml')
        kml_file = doc_kml

    domTree = parse(kml_file)
    if os.path.isfile(doc_kml):
        os.remove(doc_kml)
    rootNode = domTree.documentElement
    Placemarks = rootNode.getElementsByTagName('Placemark')

    polygon_dict = {}
    j = 0

    for Placemark in Placemarks:
        name = Placemark.getElementsByTagName('name')[0].childNodes[0].data
        ploygon = Placemark.getElementsByTagName('Polygon')[0]
        outerBoundaryIs = ploygon.getElementsByTagName('outerBoundaryIs')[0]
        LinearRing = outerBoundaryIs.getElementsByTagName('LinearRing')[0]
        coordinates = LinearRing.getElementsByTagName(
            'coordinates')[0].childNodes[0].data
        lon_lat = [i.split(',')[0:2] for i in coordinates.strip().split(' ')]
        polygon_dict[name + '-' + str(j)] = np.asarray(lon_lat,
                                                       dtype='float64')
        j += 1
    return polygon_dict


def get_lon_lat(polygon):
    lon_min = np.min(polygon[:, 0])
    lon_max = np.max(polygon[:, 0])
    lat_min = np.min(polygon[:, 1])
    lat_max = np.max(polygon[:, 1])

    return lon_min, lon_max, lat_min, lat_max


def filter_data(orig_data, polygon, data_flag, number_flag='t'):
    def filter_ts(orig_data, lonlat):
        lon_min, lon_max, lat_min, lat_max = lonlat
        lon_data = orig_data[1:, 1]
        lat_data = orig_data[1:, 2]
        lon_index = ((lon_data > lon_min) == (lon_data < lon_max))
        lat_index = ((lat_data > lat_min) == (lat_data < lat_max))
        index = (lon_index & lat_index)
        first_row = orig_data[0, :]
        left_row = orig_data[1:, :]
        cutted_data = np.vstack((first_row, left_row[index, :]))

        return cutted_data

    def filter_vel(orig_data, lonlat):
        lon_min, lon_max, lat_min, lat_max = lonlat
        lon_data = orig_data[:, 1]
        lat_data = orig_data[:, 2]
        lon_index = ((lon_data > lon_min) == (lon_data < lon_max))
        lat_index = ((lat_data > lat_min) == (lat_data < lat_max))
        index = (lon_index & lat_index)
        cutted_data = orig_data[index, :]

        return cutted_data

    lonlat = get_lon_lat(polygon)

    if number_flag == 't':
        if data_flag == 't':
            return filter_ts(orig_data, lonlat)
        else:
            return filter_vel(orig_data, lonlat)
    else:
        if data_flag == 't':
            number = np.arange(-1, orig_data.shape[0] - 1).reshape((-1, 1))
            orig_data = np.hstack((number, orig_data))
            return filter_ts(orig_data, lonlat)
        else:
            number = np.arange(0, orig_data.shape[0]).reshape((-1, 1))
            orig_data = np.hstack((number, orig_data))
            return filter_vel(orig_data, lonlat)


def cut_vel_single(kml_file, vel_file, number_flag='t'):
    out_vel_file = os.path.splitext(vel_file)[0] + '_cut.txt'
    polygon_dict = kml2polygon_dict(kml_file)
    print('loading...')
    vel = np.loadtxt(vel_file)
    for _, polygon in polygon_dict.items():
        filter_vel = filter_data(vel, polygon, 'v', number_flag)
        out_data = np.arange(vel.shape[1])
        for line in filter_vel:
            if inpolygon(line[1:3], polygon):
                if number_flag == 't':
                    out_data = np.vstack((out_data, line))
                else:
                    out_data = np.vstack((out_data, line[1:4]))
        if out_data.size > vel.shape[1]:
            np.savetxt(out_vel_file, out_data[1:, :], fmt='%4f')
    print('all done, enjoy it.\n')


def cut_ts_single(kml_file, ts_file, number_flag='t'):
    out_ts_file = os.path.splitext(ts_file)[0] + '_cut.txt'
    polygon_dict = kml2polygon_dict(kml_file)
    print('loading...')
    data = np.loadtxt(ts_file)
    for _, polygon in polygon_dict.items():
        filter_ts = filter_data(data, polygon, 't', number_flag)
        out_data = data[0, :]
        for line in filter_ts[1:, :]:
            if inpolygon(line[1:3], polygon):
                if number_flag == 't':
                    out_data = np.vstack((out_data, line))
                else:
                    out_data = np.vstack((out_data, line[1:]))
        if out_data.size > filter_ts[1:, :].shape[1]:
            np.savetxt(out_ts_file, out_data, fmt='%4f')
    print('all done, enjoy it.\n')


def cut_vel_multi(kml_file, vel_file, number_flag='t'):
    print('loading...')
    vel = np.loadtxt(vel_file)
    polygon_dict = kml2polygon_dict(kml_file)
    num = len(polygon_dict)
    i = 0
    for name, polygon in polygon_dict.items():
        i += 1
        print(f'\rProcessing: {i}/{num}', end=" ", flush=True)
        out_data = np.arange(vel.shape[1])
        filter_vel = filter_data(vel, polygon, 'v', number_flag)
        for line in filter_vel:
            if inpolygon(line[1:3], polygon):
                if number_flag == 't':
                    out_data = np.vstack((out_data, line))
                else:
                    out_data = np.vstack((out_data, line[1:4]))
        out_file = name + '-vel.txt'
        if out_data.size > vel.shape[1]:
            np.savetxt(out_file, out_data[1:, :], fmt='%4f')
    print(f'\rProcessed: {i}/{num}, enjoy it.', end=" ", flush=True)
    print('\n')


def cut_ts_multi(kml_file, ts_file, number_flag='t'):
    print('loading...')
    data = np.loadtxt(ts_file)
    polygon_dict = kml2polygon_dict(kml_file)
    num = len(polygon_dict)
    i = 0
    for name, polygon in polygon_dict.items():
        i += 1
        print(f'\rProcessing: {i}/{num}', end="", flush=True)
        out_data = data[0, :]
        filter_ts = filter_data(data, polygon, 't', number_flag)
        for line in filter_ts[1:, :]:
            if inpolygon(line[1:3], polygon):
                if number_flag == 't':
                    out_data = np.vstack((out_data, line))
                else:
                    out_data = np.vstack((out_data, line[1:]))
        out_file = name + '-ts.txt'
        if out_data.size > filter_ts[1:, :].shape[1]:
            np.savetxt(out_file, out_data, fmt='%4f')
    print(f'\rProcessed: {i}/{num}, enjoy it.', end=" ", flush=True)
    print('\n')

## read velocity

In [ ]:
os.chdir('/media/ly/file/sc_prj/cut')

vel_file = 'geo_velocity.h5'
mask_file = 'geo_maskTempCoh.h5'

read_vel(vel_file, mask_file, ds_factor=3, out_vel_file='vel.txt')

## read time-series and velocity

In [ ]:
os.chdir('/media/ly/3TB/sc_prj_new/asc2_meks_sbas/1/geo')

ts_file = 'geo_timeseries_tropHgt_ramp_demErr.h5'
vel_file = 'geo_velocity.h5'
mask_file = 'geo_maskTempCoh.h5'

out_ts, date = read_vel_ts(ts_file, vel_file, mask_file, ds_factor=3, out_vel_file='vel.txt', out_ts_file='ts.txt')

In [ ]:
!python3 make_kmz_ts.py ts.txt ts.kmz

In [ ]:
!python3 make_kmz_vel.py vel.txt vel.kmz

## cut velocity

### single

In [ ]:
from multiprocessing import Process
if __name__ == "__main__":
    p = Process(target=cut_vel_single,args=('cut.kmz', 'vel.txt', 't'))
    p.start()
    p.join()

In [ ]:
!python3 make_kmz_vel.py vel_cut.txt vel_cut -s 0.6

### multiple

In [ ]:
from multiprocessing import Process
if __name__ == "__main__":
    p = Process(target=cut_vel_multi,args=('cut.kmz', 'vel.txt', 't'))
    p.start()
    p.join()

In [ ]:
import glob
files = glob.glob('Untitled Polygon-*-vel.txt')
for file in files:
    print(f'\rProcessing: {files.index(file) + 1}/{len(files)}', end=" ", flush=True)
    cmd_str = f"python3 make_kmz_vel.py '{file}' '{file[:-4]}' -s 0.6"
    os.system(cmd_str)
print(f'\rProcessed: {len(files)}/{len(files)}', end=" ", flush=True)

## cut time-series

### single

In [ ]:
from multiprocessing import Process
if __name__ == "__main__":
    p = Process(target=cut_ts_single,args=('cut.kmz', 'ts.txt', 't'))
    p.start()
    p.join()

In [ ]:
!python3 make_kmz_ts.py ts_cut.txt ts_cut -s 0.6

### multiple

In [ ]:
from multiprocessing import Process
if __name__ == "__main__":
    p = Process(target=cut_ts_multi,args=('cut.kmz', 'ts.txt', 't'))
    p.start()
    p.join()

In [ ]:
import glob
files = glob.glob('Untitled Polygon-*-ts.txt')
for file in files:
    print(f'\rProcessing: {files.index(file) + 1}/{len(files)}', end=" ", flush=True)
    cmd_str = f"python3 make_kmz_ts.py '{file}' '{file[:-4]}' -s 0.6"
    os.system(cmd_str)
print(f'\rProcessed: {len(files)}/{len(files)}', end=" ", flush=True)

## plot time-series displacement

In [ ]:
num_list = [1000]
plot_displacement(num_list, out_ts, date, aspect=0.4, figsize=(30, 15), y_lim=[-100, 50], fig_name=None)